In [1]:
from sagemaker.amazon.amazon_estimator import get_image_uri
import sagemaker
import boto3

In [2]:
bucket = 'titanic-dataset-2020-06'
prefix = "samples"
s3_url = f's3://{bucket}/{prefix}'

docker_image_name = get_image_uri(boto3.Session().region_name, 'xgboost', repo_version='1.0-1')
role = sagemaker.get_execution_role()

xgb_hyperparams = {
    "num_round": 100,
    "eval_metric": "auc",
    "objective": "binary:logistic",
    "early_stopping_rounds": 20,
}

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


In [3]:
estimator = sagemaker.estimator.Estimator(
    image_name=docker_image_name,
    role=role,
    hyperparameters=xgb_hyperparams,
    train_instance_count=1, 
    train_instance_type="ml.m5.large", 
    train_volume_size=1, 
    output_path=f"{s3_url}/output", 
    sagemaker_session=sagemaker.Session(),
    train_use_spot_instances=True,
    checkpoint_local_path="/root/checkpoints/",
    checkpoint_s3_uri=f"{s3_url}/checkpoints",
    train_max_wait=300,
    train_max_run=300,
)

In [4]:
estimator.fit(
    {
        'train': sagemaker.session.s3_input(f"{s3_url}/train", content_type ='csv'), 
        'validation': sagemaker.session.s3_input(f"{s3_url}/test", content_type ='csv')
    }
)

2020-06-14 22:33:28 Starting - Starting the training job...
2020-06-14 22:33:31 Starting - Launching requested ML instances......
2020-06-14 22:34:45 Starting - Preparing the instances for training.........
2020-06-14 22:36:28 Downloading - Downloading input data
2020-06-14 22:36:28 Training - Downloading the training image...
2020-06-14 22:36:55 Uploading - Uploading generated training model
2020-06-14 22:36:55 Completed - Training job completed
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter eval_metric value auc to Json.
Returning the value itself
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determi

In [82]:
estimator.fit(
    {
        'train': sagemaker.session.s3_input(f"{s3_url}/train", content_type ='csv'), 
        'validation': sagemaker.session.s3_input(f"{s3_url}/test", content_type ='csv')
    }
)

2020-06-11 16:50:03 Starting - Starting the training job...
2020-06-11 16:50:05 Starting - Launching requested ML instances......
2020-06-11 16:51:24 Starting - Preparing the instances for training......
2020-06-11 16:52:08 Downloading - Downloading input data...
2020-06-11 16:53:04 Training - Training image download completed. Training in progress.
2020-06-11 16:53:04 Uploading - Uploading generated training model.INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter eval_metric value auc to Json.
Returning the value itself
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ',

In [19]:
from sagemaker.predictor import csv_serializer

In [22]:
pr = estimator.deploy(
    initial_instance_count=1, 
    instance_type='ml.c4.xlarge',
    endpoint_name='titanic-simple-estimator-endpoint2',
    serializer=csv_serializer
)

---------------!

In [26]:
!head titanic_test_sample.csv

0,3,32.0,8.3625,0,0
1,1,15.0,211.3375,0,1
0,1,-1.0,26.55,0,0
1,3,38.0,31.3875,1,5
1,1,53.0,51.4792,2,0
0,3,-1.0,8.05,0,0
0,1,44.0,90.0,2,0
1,1,40.0,153.4625,0,0
0,3,24.0,8.85,0,0
0,3,44.0,8.05,0,0


In [28]:
import pandas as pd

In [30]:
df = pd.read_csv("titanic_test_sample.csv", index_col=None, header=None)

In [32]:
df.head()

,0,1,2,3,4,5
0,0,3,32.0,8.3625,0,0
1,1,1,15.0,211.3375,0,1
2,0,1,-1.0,26.5500,0,0
3,1,3,38.0,31.3875,1,5
4,1,1,53.0,51.4792,2,0


In [34]:
df.values[:5, 1:]

array([[  3.    ,  32.    ,   8.3625,   0.    ,   0.    ],
       [  1.    ,  15.    , 211.3375,   0.    ,   1.    ],
       [  1.    ,  -1.    ,  26.55  ,   0.    ,   0.    ],
       [  3.    ,  38.    ,  31.3875,   1.    ,   5.    ],
       [  1.    ,  53.    ,  51.4792,   2.    ,   0.    ]])

In [77]:
pr.predict(np.zeros((1, 5)))

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (502) from model with message "<html>
<head><title>502 Bad Gateway</title></head>
<body bgcolor="white">
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.10.3 (Ubuntu)</center>
</body>
</html>
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/titanic-simple-estimator-endpoint2 in account 410029928830 for more information.

In [58]:
import boto3

client = boto3.client('sagemaker-runtime')

In [70]:
response = client.invoke_endpoint(
    EndpointName='titanic-simple-estimator-endpoint2', 
    ContentType="text/csv",
    Body="""
0,0,0,0,0"""
)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (502) from model with message "<html>
<head><title>502 Bad Gateway</title></head>
<body bgcolor="white">
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.10.3 (Ubuntu)</center>
</body>
</html>
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/titanic-simple-estimator-endpoint2 in account 410029928830 for more information.

In [6]:
from sagemaker import RealTimePredictor

In [74]:
import numpy as np

In [72]:
predictor = RealTimePredictor(endpoint='titanic-simple-estimator-endpoint2')

In [76]:
predictor.predict(np.zeros((1, 5)))

ParamValidationError: Parameter validation failed:
Invalid type for parameter Body, value: [[0. 0. 0. 0. 0.]], type: <class 'numpy.ndarray'>, valid types: <class 'bytes'>, <class 'bytearray'>, file-like object

In [12]:
!head titanic_train_sample.csv

0,2,-1.0,0.0,0,0
0,1,47.0,52.0,0,0
0,2,19.0,10.5,0,0
0,2,25.0,13.0,0,0
0,3,30.0,7.25,0,0
1,1,-1.0,79.2,0,0
0,1,2.0,151.55,1,2
0,3,-1.0,8.1375,0,0
1,3,4.0,22.025,0,2
0,3,-1.0,69.55,8,2


In [83]:
estimator = sagemaker.estimator.Estimator(
    image_name=docker_image_name,
    role=role,
    hyperparameters=xgb_hyperparams,
    train_instance_count=3, 
    train_instance_type="ml.m5.large", 
    train_volume_size=1, 
    output_path=f"{s3_url}/output", 
    sagemaker_session=sagemaker.Session(),
    train_use_spot_instances=True,
    checkpoint_local_path="/root/checkpoints/",
    checkpoint_s3_uri=f"{s3_url}/checkpoints",
    train_max_wait=300,
    train_max_run=300,
)

In [84]:
estimator.fit(
    {
        'train': sagemaker.session.s3_input(f"{s3_url}/train", content_type ='csv'), 
        'validation': sagemaker.session.s3_input(f"{s3_url}/test", content_type ='csv')
    }
)

2020-06-11 16:54:07 Starting - Starting the training job...
2020-06-11 16:54:09 Starting - Launching requested ML instances......
2020-06-11 16:55:27 Starting - Preparing the instances for training.........
2020-06-11 16:56:40 Downloading - Downloading input data...
2020-06-11 16:57:28 Training - Training image download completed. Training in progress.INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter eval_metric value auc to Json.
Returning the value itself
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[16:57:29] 7

In [96]:
predictor = sagemaker.predictor.RealTimePredictor(endpoint="titanic-endpoint-1")

In [107]:
predictor.predict(
    """w9,3w,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27wwe,ww0,w2w,347w742,11.1ff3qw33,,S,""", 
    initial_args={'ContentType': "text/csv"}
)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from container-1 with message "y contains previously unseen labels: '3w'". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/titanic-endpoint-1 in account 410029928830 for more information.